In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import json
import pandas as pd

#year = 
BASE_URL = 'https://www.boxofficemojo.com'
rank_PATH = '/year/world/'


In [ ]:
##### Make Soup for courses page

def load_cache1():
    try:
        cache_file = open(CACHE_FILE_NAME1, 'r')
        cache_file_contents = cache_file.read()
        cache = json.loads(cache_file_contents)
        cache_file.close()
    except:
        cache = {}
    return cache


def save_cache1(cache):
    cache_file = open(CACHE_FILE_NAME1, 'w')
    contents_to_write = json.dumps(cache)
    cache_file.write(contents_to_write)
    cache_file.close()


def make_url_request_using_cache1(url, cache):
    if (url in cache.keys()): # the url is our unique key
        print("Using cache")
        return cache[url]
    else:
        print("Fetching")
        #time.sleep(1)
        response = requests.get(url)
        cache[url] = response.text
        save_cache1(cache)
        return cache[url]

In [ ]:
CACHE_FILE_NAME1 = 'cacheMovie_Scrape.json'
CACHE_DICT1 = {}

In [ ]:
##### Make Soup for courses page

def load_cache2():
    try:
        cache_file = open(CACHE_FILE_NAME2, 'r')
        cache_file_contents = cache_file.read()
        cache = json.loads(cache_file_contents)
        cache_file.close()
    except:
        cache = {}
    return cache


def save_cache2(cache):
    cache_file = open(CACHE_FILE_NAME2, 'w')
    contents_to_write = json.dumps(cache)
    cache_file.write(contents_to_write)
    cache_file.close()


def make_url_request_using_cache2(url, cache):
    if (url in cache.keys()): # the url is our unique key
        print("Using cache")
        return cache[url]
    else:
        print("Fetching")
        #time.sleep(1)
        response = requests.get(url)
        cache[url] = response.text
        save_cache2(cache)
        return cache[url]

In [ ]:
CACHE_FILE_NAME2 = 'cacheDetail_Scrape.json'
CACHE_DICT2 = {}

In [ ]:
rank_url = BASE_URL + rank_PATH

for i in range(2021,2022):
    Genre = []
    URL = []
    Rank = []
    Name = []
    Year = []
    Length = []
    Country = []
    Rating = []
    Description = []
    url = rank_url + str(i)

    #page = requests.get(url)
    CACHE_DICT1 = load_cache1()
    url_text = make_url_request_using_cache1(url, CACHE_DICT1)
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')
    
    movie_tds = soup.find_all('td', class_='a-text-left mojo-field-type-release_group')
    rank = 1
    for movie_td in movie_tds:
        ###extract course details url
        movie_link_tag = movie_td.find('a', class_="a-link-normal")
        movie_details_path = movie_link_tag['href']
        movie_details_url = BASE_URL + movie_details_path
        URL.append(movie_details_url)
        #print(movie_details_url)
        
        ####CRAWL TO THE COURSE DETAIL PAGE AND EXTRACT:
        #<div class="a-section a-spacing-none"><h1 class="a-size-extra-large">The Eight Hundred<span class="a-size-large a-color-secondary"> (2020)</span></h1><span class="a-size-medium">From the acclaimed filmmaker behind Mr. Six comes a riveting war epic. In 1937, eight hundred Chinese soldiers fight under siege from a warehouse in the middle of the Shanghai battlefield, completely surrounded by the Japanese army.</span></div>
        CACHE_DICT2 = load_cache2()
        responseDetail = make_url_request_using_cache2(movie_details_url, CACHE_DICT2)
        #response = requests.get(movie_details_url)
        soupDetail = BeautifulSoup(responseDetail, 'html.parser')
        details = soup.find_all('span')#, class_='a-text-left mojo-field-type-release_group')
        
        
        Rank.append(rank)
        print('Rank: ', rank)
        rank += 1
        movie_name = soupDetail.find('h1', class_='a-size-extra-large')#.find('h1')
        #number_name2 = number_name.find('h1')
        Name.append(movie_name.text.strip())
        #print('Movie name: ',movie_name.text.strip())
        years = i
        Year.append(years)
        #print('Release Year: ', years)
        
        #<div class="a-section a-spacing-none"><span>Running Time</span><span>2 hr 10 min</span></div>
        #<span>2 hr 10 min</span>
        #<div class="a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile"><div class="a-section a-spacing-none"><span>Earliest Release Date</span><span>January 15, 2021
            #(China)</span></div><div class="a-section a-spacing-none"><span>Running Time</span><span>2 hr 10 min</span></div><div class="a-section a-spacing-none"><span>Genres</span><span>Drama
    
        
        #length = soupDetail.find('div', class_='a-section a-spacing-none mojo-summary-values mojo-hidden-from-mobile').find(class_="a-section a-spacing-none")
        #print(length)
        
        baseurl = "https://itunes.apple.com/search"
        parameter_dictionary = {'term':movie_name.text.strip()}
        
        resp = requests.get(baseurl, parameter_dictionary)
        #json_str = resp.text
        if resp.status_code != '204':
            Results_Dictionary = resp.json()
        else:
            Results_Dictionary=''
        
        #print('Movie Length(mins): ', round((Results_Dictionary["results"][0]["trackTimeMillis"]/60000)))
        
               
                    
        if Results_Dictionary["resultCount"] > 0 and Results_Dictionary !='':
            #index = 0
            for re in Results_Dictionary["results"]:
                if 'kind' in re:
        
                    #break
                    if re["kind"] == 'feature-movie':
                        Genre.append(re['primaryGenreName'])
                        Length.append(round((re["trackTimeMillis"]/60000)))
                        Rating.append(re['contentAdvisoryRating'])
                        Country.append(re['country'])
                        #print(re)
                    
                        '''print('Genre: ', re['primaryGenreName'])
                        print('Movie Length(mins): ', round((re["trackTimeMillis"]/60000)))
                        print('Rating: ', re['contentAdvisoryRating'])
                        print('Country: ', re['country'])'''
                        break
                    else:
                        Genre.append('Unknown')
                        Length.append('Unknown')
                        Rating.append('Unknown')
                        Country.append('Unknown')
                        '''print('Genre: Unknown')
                        print('Movie Length(mins): Unknown')
                        print('Rating: Unknown')
                        print('Country: Unknown')'''
                        break
                else:
                    Genre.append('Unknown')
                    Length.append('Unknown')
                    Rating.append('Unknown')
                    Country.append('Unknown')
                    '''print('Genre: Unknown')
                    print('Movie Length(mins): Unknown')
                    print('Rating: Unknown')
                    print('Country: Unknown')'''
                    break
        else: 
            Genre.append('Unknown')
            Length.append('Unknown')
            Rating.append('Unknown')
            Country.append('Unknown')
            '''print('Genre: Unknown')
            print('Movie Length(mins): Unknown')
            print('Rating: Unknown')
            print('Country: Unknown')'''
        
        
                 
            
        #print('Movie Length: ', Results_Dictionary[1]["wrapperType"]["trackTimeMillis"]/60000)
        #print("")
        #<div class="a-fixed-left-grid-col a-col-right" style="padding-left:0%;float:left;"><h1 class="a-size-extra-large">The Eight Hundred</h1><p class="a-size-medium">From the acclaimed filmmaker behind Mr. Six comes a riveting war epic. In 1937, eight hundred Chinese soldiers fight under siege from a warehouse in the middle of the Shanghai battlefield, completely surrounded by the Japanese army.</p></div>
        #<div class="a-fixed-left-grid-col a-col-right" style="padding-left:0%;float:left;"><div class="a-section a-spacing-none"><h1 class="a-size-extra-large">The Eight Hundred<span class="a-size-large a-color-secondary"> (2020)</span></h1><span class="a-size-medium">From the acclaimed filmmaker behind Mr. Six comes a riveting war epic. In 1937, eight hundred Chinese soldiers fight under siege from a warehouse in the middle of the Shanghai battlefield, completely surrounded by the Japanese army.</span></div></div>
        
        #print("")
        #<span class="a-size-medium">From the acclaimed filmmaker behind Mr. Six comes a riveting war epic. In 1937, eight hundred Chinese soldiers fight under siege from a warehouse in the middle of the Shanghai battlefield, completely surrounded by the Japanese army.</span>
        movie_desc = soupDetail.find('div', class_='a-fixed-left-grid-col a-col-right').find(class_="a-size-medium")
       
        #for x in movie_desc:
        #    print(x.text.strip())
        if movie_desc is None:
            Description.append('None')
            #print('Short Description: None')
        else:
            Description.append(movie_desc.text.strip())
            #print('Short Description: ', movie_desc.text.strip())
        #print("")

    
        print('-' * 50) # seperator 

In [ ]:
df = pd.DataFrame({'URL': URL,
                   'Rank': Rank,
                   'Name': Name,
                   'Year': Year,
                   'Length': Length,
                   'Country': Country,
                   'Rating': Rating,
                   'Genre': Genre,
                   'Description': Description})

In [ ]:
df